## Goal
We want to explore how better is the performance of the system of generating a draft with a multiple step process rather than a single step process

### Initial Variables

In [1]:
import sys 
import json
sys.path.append("../../")
from backend.LLM.AnythingLLM_client import AnythingLLMClient
from backend.Generater import Generater
from backend.LLM.OllamaLLM import OllamaAI
from backend.database.schemas import Email


anyllm_client = AnythingLLMClient("http://localhost:3001/api", "3WMNAPZ-GYH4RBE-M67SR00-7Y7KYEF")
ollama_client = OllamaAI('http://localhost:11434', 'llama3:instruct')
generator = Generater(ollama_client, anyllm_client)
path_to_enquiry = "../../dataset/fake_email_1.json"
with open(path_to_enquiry, "r") as f:
    enquiry = json.load(f)

enquiry_text = enquiry["email"]
subject = enquiry_text.split("\n\n")[0]
body = '\n'.join(enquiry_text.split("\n\n")[1:])

Performing health check on http://localhost:11434
Ollama is running
Health check passed


### Single Step Process

In [2]:


prompt = f""" You are David Frischer, the program administrator for the UCL Software Engineering program.
        You have received an email from a student asking the following questions:
        Subject: {subject}
        Body: {body}
        
        Please draft a response to the student's email.
        
        Email:
        {enquiry_text}
        """
        

response = anyllm_client.chat_with_workspace("single-shot", prompt)
print(response)

{'id': '72b718d6-a866-4032-a56c-9fe240942d6d', 'type': 'textResponse', 'close': True, 'error': None, 'chatId': 600, 'textResponse': "Subject: Application Questions for MSc in Software Engineering at UCL\n\nDear John,\n\nThank you for your email and for considering the Master's degree program in Software Engineering at University College London (UCL). I'm happy to help clarify a few essential details before submitting your application.\n\n1. English Language Requirements:\nAs an international student like yourself, you will need to take a recognized English language proficiency test (e.g., IELTS or TOEFL) prior to applying for the MSc program in Software Engineering. The required evidence should be included with the complete application; however, you can submit your application before taking a test. If unsatisfactory or no evidence of English language proficiency has been provided with the application, admission will be conditional upon the provision of such evidence.\n\n2. Part-time Em

### Response format

In [3]:
textResponse = response['textResponse']
print(textResponse)

Subject: Application Questions for MSc in Software Engineering at UCL

Dear John,

Thank you for your email and for considering the Master's degree program in Software Engineering at University College London (UCL). I'm happy to help clarify a few essential details before submitting your application.

1. English Language Requirements:
As an international student like yourself, you will need to take a recognized English language proficiency test (e.g., IELTS or TOEFL) prior to applying for the MSc program in Software Engineering. The required evidence should be included with the complete application; however, you can submit your application before taking a test. If unsatisfactory or no evidence of English language proficiency has been provided with the application, admission will be conditional upon the provision of such evidence.

2. Part-time Employment:
International students like yourself are permitted to work part-time during their studies at UCL. However, please note that you shou

## Mutiple Step Process

In [2]:

new_email = Email(
  body=body,
  subject=subject,
  id=-1,
)
path_output = "response.json"
results = generator.single_run_reply_to_email(new_email, path_output)

Total Sim Distance: 0.39253494143486023
Total Sim Distance: 0.5035082548856735
Total Sim Distance: 0.41801171004772186


In [9]:
print(results["generated_draft_email"])

Subject: Re: Application Questions for MSc in Software Engineering at UCL

Dear John,

Thank you for your interest in our Master's degree program in Software Engineering at University College London (UCL). I'm happy to help clarify some essential details before submitting your application.

**Regarding your first question:** As an international student, you do need to demonstrate English language proficiency. Please check the entry requirements section of your programme's prospectus page or visit our English language requirements webpage for specific information on English language tests and scores required.

**For your second question:** As an international student at UCL, you are allowed to work part-time. You can find more information on your fee status and the implications of working during your studies by visiting our Student Fee Status webpages. Additionally, please check with your academic department or supervisor for any specific guidelines or restrictions related to employment

## Generate for multiple drafts